In [ ]:
import json
from tqdm import tqdm
from os import walk

from scipy import stats
import numpy as np

from scipy import stats
from langdetect import detect

In [ ]:
data_path = ".LLM_Scored/run_config/"
filenames = list(set(next(walk(data_path), (None,None,[]))[2]))

495


In [3]:
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

In [4]:
def count_tokens(recette, thrs=20):
    list_recette = recette.split()
    if len(list_recette) > thrs:
        return True
    else:
        return False

In [ ]:
#Change layer names based on the analyzed layers you kept in measuring novelty
layer_list = ['layer_original', 'layer_1', 'layer_23', 'layer_-2', 'layer_-1', 'layer_last']
tradi_list = ['authenticity', 'tradition', 'prototype']
novel_list = ['novelty', 'uniquness', 'newness', 'difference', 'surprise', 'creativity', 'originality']

# Measuring divergence between same coutries from last layers first

In [8]:
for layer in layer_list:
    #For the country of origin
    human_new_orig, LLM_new_orig, human_new_varia, LLM_new_varia = [], [], [], []
    human_uniq_orig, LLM_uniq_orig, human_uniq_varia, LLM_uniq_varia = [], [], [], []
    human_diff_orig, LLM_diff_orig, human_diff_varia, LLM_diff_varia = [], [], [], []
    human_nsurp_orig, LLM_nsurp_orig, human_nsurp_varia, LLM_nsurp_varia = [], [], [], []
    human_dsurp_orig, LLM_dsurp_orig, human_dsurp_varia, LLM_dsurp_varia = [], [], [], []
    for k in tqdm(range(len(filenames))):
        recette = filenames[k]
        file_path = data_path + recette
        with open(file_path) as json_file:
            recipe_dict = json.load(json_file)

        country_orig = recipe_dict['Country'].lower()
        train_recipe_ids = list(recipe_dict['Train_Variations'].keys())
        for i in range(len(train_recipe_ids)):
            recipe = recipe_dict['Train_Variations'][train_recipe_ids[i]]
            if recipe['country'].lower() == country_orig:
                human_new_orig.append(recipe[layer]['newness_div'])
                human_uniq_orig.append(recipe[layer]['uniq_dist'])
                human_diff_orig.append(recipe[layer]['diff_local'])
                human_nsurp_orig.append(recipe[layer]['new_surprise'])
                human_nsurp_orig.append(recipe[layer]['dist_surprise'])
                
        valid_recipe_ids = list(recipe_dict['Valid_Variations'].keys())
        for i in range(len(valid_recipe_ids)):
            recipe = recipe_dict['Valid_Variations'][valid_recipe_ids[i]]
            if recipe['country'].lower() == country_orig:
                human_new_orig.append(recipe[layer]['newness_div'])
                human_uniq_orig.append(recipe[layer]['uniq_dist'])
                human_diff_orig.append(recipe[layer]['diff_local'])
                human_nsurp_orig.append(recipe[layer]['new_surprise'])
                human_dsurp_orig.append(recipe[layer]['dist_surprise'])

        test_recipe_ids = list(recipe_dict['Test_Variations'].keys())
        for i in range(len(test_recipe_ids)):
            recipe = recipe_dict['Test_Variations'][test_recipe_ids[i]]
            if recipe['country'].lower() == country_orig:
                human_new_orig.append(recipe[layer]['newness_div'])
                human_uniq_orig.append(recipe[layer]['uniq_dist'])
                human_diff_orig.append(recipe[layer]['diff_local'])
                human_nsurp_orig.append(recipe[layer]['new_surprise'])
                human_dsurp_orig.append(recipe[layer]['dist_surprise'])
                
    ### KEEP THIS PART WHEN YOU WANT TO OBSERVE THE CULTURAL DISTANCE -- TO WHICH TYPE OF COUNTRY IT CORRESPOND?? 
        if 'same_country_novelty' in recipe_dict['LLM_gen']:
            llm_recipe_ids = list(recipe_dict['LLM_gen']['same_country_novelty'].keys())
            for i in range(len(llm_recipe_ids)):
                tradi_text = llm_recipe_ids[i].split('_')[-1]
                #if tradi_text in tradi_list: 
                recipe = recipe_dict['LLM_gen']['same_country_novelty'][llm_recipe_ids[i]]
                is_eng = is_english(recipe['Instructions'])
                if is_eng:
                    count_ = count_tokens(recipe['Instructions'])
                    if count_:
                        LLM_new_orig.append(recipe[layer]['newness_div'])
                        LLM_uniq_orig.append(recipe[layer]['uniq_dist'])
                        LLM_diff_orig.append(recipe[layer]['diff_local'])
                        LLM_nsurp_orig.append(recipe[layer]['new_surprise'])
                        LLM_dsurp_orig.append(recipe[layer]['dist_surprise'])

    print('===============================')
    print('Analyzed Layer : ', layer)
    x0 = np.array(human_new_orig)
    y0 = np.array(LLM_new_orig)
    x1 = np.array(human_uniq_orig)
    y1 = np.array(LLM_uniq_orig)
    x2 = np.array(human_diff_orig)
    y2 = np.array(LLM_diff_orig)
    x3 = np.array(human_nsurp_orig)
    y3 = np.array(LLM_nsurp_orig)
    x4 = np.array(human_dsurp_orig)
    y4 = np.array(LLM_dsurp_orig)

    # Step 1: Mean values
    mean_x0, mean_y0 = np.mean(x0), np.mean(y0)
    print("Mean Human Newness:", mean_x0, "Mean LLM Newness:", mean_y0)
    mean_x1, mean_y1 = np.nanmean(x1), np.nanmean(y1)
    print("Mean Human Uniq:", mean_x1, "Mean LLM Uniq:", mean_y1)
    mean_x2, mean_y2 = np.nanmean(x2), np.nanmean(y2)
    print("Mean Human Difference:", mean_x2, "Mean LLM Difference:", mean_y2)
    mean_x3, mean_y3 = np.mean(x3), np.mean(y3)
    print("Mean Human Newn Surprsie:", mean_x3, "Mean Newn Surprsie:", mean_y3)
    mean_x4, mean_y4 = np.mean(x4), np.mean(y4)
    print("Mean Human Dist Surprise:", mean_x4, "Mean LLM Dist Surprise:", mean_y4)

    # Step 2a: Independent t-test (Welch’s by default with equal_var=False)
    t_stat0, p_val_t0 = stats.ttest_ind(x0, y0, equal_var=False)
    print("t-test p-value NEwness:", p_val_t0)
    t_stat1, p_val_t1 = stats.ttest_ind(x1, y1, equal_var=False)
    print("t-test p-value Uniq:", p_val_t1)
    t_stat2, p_val_t2 = stats.ttest_ind(x2, y2, equal_var=False)
    print("t-test p-value Difference:", p_val_t2)
    t_stat3, p_val_t3 = stats.ttest_ind(x3, y3, equal_var=False)
    print("t-test p-value New Surprise:", p_val_t3)
    t_stat4, p_val_t4 = stats.ttest_ind(x4, y4, equal_var=False)
    print("t-test p-value Dist Surpr:", p_val_t4)

    # Step 2b: Mann–Whitney U test
    u_stat0, p_val_u0 = stats.mannwhitneyu(x0, y0, alternative='two-sided')
    print("Mann–Whitney p-value NEwness:", p_val_u0)
    # Step 2b: Mann–Whitney U test
    u_stat1, p_val_u1 = stats.mannwhitneyu(x1, y1, alternative='two-sided')
    print("Mann–Whitney p-value Uniq:", p_val_u1)
    # Step 2b: Mann–Whitney U test
    u_stat2, p_val_u2 = stats.mannwhitneyu(x2, y2, alternative='two-sided')
    print("Mann–Whitney p-value Difference:", p_val_u2)
    # Step 2b: Mann–Whitney U test
    u_stat3, p_val_u3 = stats.mannwhitneyu(x3, y3, alternative='two-sided')
    print("Mann–Whitney p-value New Surpr:", p_val_u3)
    # Step 2b: Mann–Whitney U test
    u_stat4, p_val_u4 = stats.mannwhitneyu(x4, y4, alternative='two-sided')
    print("Mann–Whitney p-value Dist Surpr:", p_val_u4)
    print('\n')

100%|██████████| 495/495 [02:36<00:00,  3.17it/s]


Analyzed Layer :  layer_original
Mean Human Newness: 0.0027830668916057387 Mean LLM Newness: 0.006895570557044753
Mean Human Uniq: 0.38316307164409147 Mean LLM Uniq: 0.5399983494708731
Mean Human Difference: 0.5296523517382413 Mean LLM Difference: 0.9959378089332256
Mean Human Newn Surprsie: 0.5006257320591598 Mean Newn Surprsie: 0.972153433699343
Mean Human Dist Surprise: 0.3975102562490127 Mean LLM Dist Surprise: 0.5528488388328215
t-test p-value NEwness: 0.0
t-test p-value Uniq: 0.0
t-test p-value Difference: 0.0
t-test p-value New Surprise: 0.0
t-test p-value Dist Surpr: 0.0
Mann–Whitney p-value NEwness: 2.396107279482812e-54
Mann–Whitney p-value Uniq: 0.0
Mann–Whitney p-value Difference: 0.0
Mann–Whitney p-value New Surpr: 0.0
Mann–Whitney p-value Dist Surpr: 0.0




100%|██████████| 495/495 [02:34<00:00,  3.20it/s]


Analyzed Layer :  layer_1
Mean Human Newness: 0.0011134519360255714 Mean LLM Newness: 0.0047753306025139245
Mean Human Uniq: 0.5318626445270651 Mean LLM Uniq: 0.6508539090760618
Mean Human Difference: 0.8800990205575288 Mean LLM Difference: 0.9707679588792438
Mean Human Newn Surprsie: 0.6689474789555829 Mean Newn Surprsie: 0.9865923903205233
Mean Human Dist Surprise: 0.2657080184220042 Mean LLM Dist Surprise: 0.19529846689392794
t-test p-value NEwness: 0.0
t-test p-value Uniq: nan
t-test p-value Difference: 3.6672130849021064e-63
t-test p-value New Surprise: 0.0
t-test p-value Dist Surpr: 1.564782092847612e-106
Mann–Whitney p-value NEwness: 0.0
Mann–Whitney p-value Uniq: nan
Mann–Whitney p-value Difference: 7.361994995773585e-241
Mann–Whitney p-value New Surpr: 0.0
Mann–Whitney p-value Dist Surpr: 2.7541372599346232e-102




100%|██████████| 495/495 [02:35<00:00,  3.18it/s]


...

Analyzed Layer :  layer_-2
Mean Human Newness: 0.0009707152220723341 Mean LLM Newness: 0.004498949556876946
Mean Human Uniq: 0.6558507597793434 Mean LLM Uniq: 0.6558048387493313
Mean Human Difference: 0.9187385642019159 Mean LLM Difference: 0.9642131751595219
Mean Human Newn Surprsie: 0.7949461669975111 Mean Newn Surprsie: 0.9833572487678658
Mean Human Dist Surprise: 0.07433140632772477 Mean LLM Dist Surprise: 0.1873133262842399
t-test p-value NEwness: 0.0
t-test p-value Uniq: nan
t-test p-value Difference: 8.082221655431828e-23
t-test p-value New Surprise: 5.645498961894304e-240
t-test p-value Dist Surpr: 0.0
Mann–Whitney p-value NEwness: 0.0
Mann–Whitney p-value Uniq: nan
Mann–Whitney p-value Difference: 9.158351119375173e-48
Mann–Whitney p-value New Surpr: 0.0
Mann–Whitney p-value Dist Surpr: 4.042618085138591e-254




100%|██████████| 495/495 [02:35<00:00,  3.18it/s]


Analyzed Layer :  layer_-1
Mean Human Newness: 0.0012129618228646616 Mean LLM Newness: 0.00428551255940128
Mean Human Uniq: 0.49806214123126247 Mean LLM Uniq: 0.5833921262727564
Mean Human Difference: 0.8984501130125928 Mean LLM Difference: 0.969010009524323
Mean Human Newn Surprsie: 0.6953137337947763 Mean Newn Surprsie: 0.9723626361486797
Mean Human Dist Surprise: 0.39455311148186867 Mean LLM Dist Surprise: 0.4825518588014746
t-test p-value NEwness: 0.0
t-test p-value Uniq: nan
t-test p-value Difference: 1.9522661738470163e-46
t-test p-value New Surprise: 0.0
t-test p-value Dist Surpr: 4.439125065451348e-147
Mann–Whitney p-value NEwness: 0.0
Mann–Whitney p-value Uniq: nan
Mann–Whitney p-value Difference: 1.789761920416219e-170
Mann–Whitney p-value New Surpr: 0.0
Mann–Whitney p-value Dist Surpr: 1.3785588319798174e-296




100%|██████████| 495/495 [02:36<00:00,  3.15it/s]

Analyzed Layer :  layer_last
Mean Human Newness: 0.0018506114360352247 Mean LLM Newness: 0.005482207195024729
Mean Human Uniq: 0.40634365295680014 Mean LLM Uniq: 0.508727226397594
Mean Human Difference: 0.5666774297707459 Mean LLM Difference: 0.9322405550163558
Mean Human Newn Surprsie: 0.5171211364008291 Mean Newn Surprsie: 0.9005393569462083
Mean Human Dist Surprise: 0.39868023786889584 Mean LLM Dist Surprise: 0.5127719431603094
t-test p-value NEwness: 0.0
t-test p-value Uniq: nan
t-test p-value Difference: 0.0
t-test p-value New Surprise: 0.0
t-test p-value Dist Surpr: 7.395684855468558e-306
Mann–Whitney p-value NEwness: 0.0
Mann–Whitney p-value Uniq: nan
Mann–Whitney p-value Difference: 0.0
Mann–Whitney p-value New Surpr: 0.0
Mann–Whitney p-value Dist Surpr: 0.0




In [ ]:
#T-TEST BETWEEN HUMAN AND LLMs
x = np.array(human_dsurp_orig)
y = np.array(LLM_dsurp_orig)

# Step 1: Mean values
mean_x, mean_y = np.mean(x), np.mean(y)
print("Mean Human:", mean_x, "Mean LLM:", mean_y)

# Step 2a: Independent t-test (Welch’s by default with equal_var=False)
t_stat, p_val_t = stats.ttest_ind(x, y, equal_var=False)
print("t-test p-value:", p_val_t)

# Step 2b: Mann–Whitney U test
u_stat, p_val_u = stats.mannwhitneyu(x, y, alternative='two-sided')
print("Mann–Whitney p-value:", p_val_u)

Mean Human: 0.14932028862994123 Mean LLM: 0.19266642040984727
t-test p-value: 7.872046782369342e-23
Mann–Whitney p-value: 2.9896601324397093e-24


# For the variation countries

In [ ]:
#For the variation countries
for layer in layer_list:
    dict_values = {}
    for k in tqdm(range(len(filenames))):
        recette = filenames[k]
        file_path = data_path + recette
        with open(file_path) as json_file:
            recipe_dict = json.load(json_file)

        varia_key_list = list(recipe_dict['LLM_gen']['variation_novelty'].keys())
        varia_country_list = [key.split("_")[1] for key in varia_key_list]

        train_recipe_ids = list(recipe_dict['Train_Variations'].keys())
        for i in range(len(train_recipe_ids)):
            recipe = recipe_dict['Train_Variations'][train_recipe_ids[i]]
            country_t = recipe['country'].lower()
            if country_t in varia_country_list:
                if country_t not in dict_values:
                    dict_values[country_t] = {'LLM_new':[], 'Human_new': [], 'LLM_uniq':[], 'Human_uniq': [],'LLM_diff':[], 'Human_diff': [],'LLM_nsurp':[], 'Human_nsurp': [],'LLM_dsurp':[], 'Human_dsurp': [],}
                dict_values[country_t]['Human_new'].append(recipe[layer]['newness_div'])
                dict_values[country_t]['Human_uniq'].append(recipe[layer]['uniq_dist'])
                dict_values[country_t]['Human_diff'].append(recipe[layer]['diff_local'])
                dict_values[country_t]['Human_nsurp'].append(recipe[layer]['new_surprise'])
                dict_values[country_t]['Human_dsurp'].append(recipe[layer]['dist_surprise'])
                
        valid_recipe_ids = list(recipe_dict['Valid_Variations'].keys())
        for i in range(len(valid_recipe_ids)):
            recipe = recipe_dict['Valid_Variations'][valid_recipe_ids[i]]
            country_v = recipe['country'].lower()
            if country_v in varia_country_list:
                if country_v not in dict_values:
                    dict_values[country_v] = {'LLM_new':[], 'Human_new': [], 'LLM_uniq':[], 'Human_uniq': [],'LLM_diff':[], 'Human_diff': [],'LLM_nsurp':[], 'Human_nsurp': [],'LLM_dsurp':[], 'Human_dsurp': [],}
                dict_values[country_v]['Human_new'].append(recipe[layer]['newness_div'])
                dict_values[country_v]['Human_uniq'].append(recipe[layer]['uniq_dist'])
                dict_values[country_v]['Human_diff'].append(recipe[layer]['diff_local'])
                dict_values[country_v]['Human_nsurp'].append(recipe[layer]['new_surprise'])
                dict_values[country_v]['Human_dsurp'].append(recipe[layer]['dist_surprise'])

        test_recipe_ids = list(recipe_dict['Test_Variations'].keys())
        for i in range(len(test_recipe_ids)):
            recipe = recipe_dict['Test_Variations'][test_recipe_ids[i]]
            country_te = recipe['country'].lower()
            if country_te in varia_country_list:
                if country_te not in dict_values:
                    dict_values[country_te] = {'LLM_new':[], 'Human_new': [], 'LLM_uniq':[], 'Human_uniq': [],'LLM_diff':[], 'Human_diff': [],'LLM_nsurp':[], 'Human_nsurp': [],'LLM_dsurp':[], 'Human_dsurp': [],}
                dict_values[country_te]['Human_new'].append(recipe[layer]['newness_div'])
                dict_values[country_te]['Human_uniq'].append(recipe[layer]['uniq_dist'])
                dict_values[country_te]['Human_diff'].append(recipe[layer]['diff_local'])
                dict_values[country_te]['Human_nsurp'].append(recipe[layer]['new_surprise'])
                dict_values[country_te]['Human_dsurp'].append(recipe[layer]['dist_surprise'])
        ## LLMs
        for i in range(len(varia_key_list)):
            country = varia_country_list[i]
            if country not in dict_values:
                dict_values[country] = {'LLM_new':[], 'Human_new': [], 'LLM_uniq':[], 'Human_uniq': [],'LLM_diff':[], 'Human_diff': [],'LLM_nsurp':[], 'Human_nsurp': [],'LLM_dsurp':[], 'Human_dsurp': [],}
            recipe_keys = list(recipe_dict['LLM_gen']['variation_novelty'][varia_key_list[i]].keys())
            if len(recipe_keys) > 0:
                recipe = recipe_dict['LLM_gen']['variation_novelty'][varia_key_list[i]]
                ### The split for the keys novel and tradi keywords should happen here
                for key in recipe_keys:
                    is_eng = is_english(recipe[key]['Instructions'])
                    if is_eng:
                        count_ = count_tokens(recipe[key]['Instructions'])
                        if count_:
                            dict_values[country]['LLM_new'].append(recipe[key][layer]['newness_div'])
                            dict_values[country]['LLM_uniq'].append(recipe[key][layer]['uniq_dist'])
                            dict_values[country]['LLM_diff'].append(recipe[key][layer]['diff_local'])
                            dict_values[country]['LLM_nsurp'].append(recipe[key][layer]['new_surprise'])
                            dict_values[country]['LLM_dsurp'].append(recipe[key][layer]['dist_surprise'])
    
    countries = list(dict_values.keys())
    gen_mean_x0, gen_mean_y0, gen_mean_x1, gen_mean_y1, gen_mean_x2, gen_mean_y2, gen_mean_x3, gen_mean_y3, gen_mean_x4, gen_mean_y4 = [], [], [], [], [], [], [], [], [], []
    gen_ttest_pvalue0, gen_ttest_pvalue1, gen_ttest_pvalue2, gen_ttest_pvalue3, gen_ttest_pvalue4 = [], [], [], [], []
    gen_utest_pval0, gen_utest_pval1, gen_utest_pval2, gen_utest_pval3, gen_utest_pval4 = [], [], [], [], []
    for country in countries:
        x0 = np.array(dict_values[country]['Human_new'])
        y0 = np.array(dict_values[country]['LLM_new'])
        x1 = np.array(dict_values[country]['Human_uniq'])
        y1 = np.array(dict_values[country]['LLM_uniq'])
        x2 = np.array(dict_values[country]['Human_diff'])
        y2 = np.array(dict_values[country]['LLM_diff'])
        x3 = np.array(dict_values[country]['Human_nsurp'])
        y3 = np.array(dict_values[country]['LLM_nsurp'])
        x4 = np.array(dict_values[country]['Human_dsurp'])
        y4 = np.array(dict_values[country]['LLM_dsurp'])

        # Step 1: Mean values
        gen_mean_x0.append(np.mean(x0))
        gen_mean_y0.append(np.mean(y0))
        gen_mean_x1.append(np.mean(x1))
        gen_mean_y1.append(np.mean(y1))
        gen_mean_x2.append(np.mean(x2))
        gen_mean_y2.append(np.mean(y2))
        gen_mean_x3.append(np.mean(x3))
        gen_mean_y3.append(np.mean(y3))
        gen_mean_x4.append(np.mean(x4))
        gen_mean_y4.append(np.mean(y4))

        # Step 2a: Independent t-test (Welch’s by default with equal_var=False)
        _, p_val_t0 = stats.ttest_ind(x0, y0, equal_var=False, nan_policy="omit")
        if not np.isnan(p_val_t0):
            gen_ttest_pvalue0.append(p_val_t0)
        _, p_val_t1 = stats.ttest_ind(x1, y1, equal_var=False, nan_policy="omit")
        if not np.isnan(p_val_t1):
            gen_ttest_pvalue1.append(p_val_t1)
        _, p_val_t2 = stats.ttest_ind(2, y2, equal_var=False, nan_policy="omit")
        if not np.isnan(p_val_t2):
            gen_ttest_pvalue2.append(p_val_t2)
        _, p_val_t3 = stats.ttest_ind(x3, y3, equal_var=False, nan_policy="omit")
        if not np.isnan(p_val_t3):
            gen_ttest_pvalue3.append(p_val_t3)
        _, p_val_t4 = stats.ttest_ind(x4, y4, equal_var=False, nan_policy="omit")
        if not np.isnan(p_val_t4):
            gen_ttest_pvalue4.append(p_val_t4)

        # Step 2b: Mann–Whitney U test
        u_stat0, p_val_u0 = stats.mannwhitneyu(x0, y0, alternative='two-sided')
        gen_utest_pval0.append(p_val_u0)
        u_stat1, p_val_u1 = stats.mannwhitneyu(x1, y1, alternative='two-sided')
        gen_utest_pval1.append(p_val_u1)
        u_stat2, p_val_u2 = stats.mannwhitneyu(x2, y2, alternative='two-sided')
        gen_utest_pval2.append(p_val_u2)
        u_stat3, p_val_u3 = stats.mannwhitneyu(x3, y3, alternative='two-sided')
        gen_utest_pval3.append(p_val_u3)
        u_stat, p_val_u4 = stats.mannwhitneyu(x4, y4, alternative='two-sided')
        gen_utest_pval4.append(p_val_u4)
    print('===================================')
    print('Layer : ', layer)
    print('Newness Human Mean : ', np.mean(gen_mean_x0))
    print('Newness LLM Mean : ', np.mean(gen_mean_y0))
    print('Newness ttest: ',np.mean(gen_ttest_pvalue0))
    print('Newness utest :',np.mean(gen_utest_pval0))
    print('----')
    print('Uniq Human Mean : ',np.nanmean(gen_mean_x1))
    print('Uniq LLM Mean : ',np.nanmean(gen_mean_y1))
    print('Uniq ttest: ',np.mean(gen_ttest_pvalue1))
    print('Uniq utest :',np.mean(gen_utest_pval1))
    print('----')
    print('Diff Human Mean : ',np.nanmean(gen_mean_x2))
    print('Diff LLM Mean : ',np.nanmean(gen_mean_y2))
    print('Diff ttest: ',np.mean(gen_ttest_pvalue2))
    print('Diff utest :',np.mean(gen_utest_pval2))
    print('----')
    print('New Surp Human Mean : ',np.nanmean(gen_mean_x3))
    print('New Surp LLM Mean : ',np.nanmean(gen_mean_y3))
    print('New Surp ttest: ',np.mean(gen_ttest_pvalue3))
    print('New Surp  utest :',np.mean(gen_utest_pval3))
    print('----')
    print('Dist Surp Human Mean : ',np.nanmean(gen_mean_x4))
    print('Dist Surp LLM Mean : ',np.nanmean(gen_mean_y4))
    print('Dist Surp ttest: ',np.mean(gen_ttest_pvalue4))
    print('Dist Surp  utest :',np.mean(gen_utest_pval4))
    print('\n')

  0%|          | 0/495 [00:00<?, ?it/s]

100%|██████████| 495/495 [18:56<00:00,  2.30s/it]
/home/mila/f/florian.carichon/.conda/envs/work0/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:586: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/home/mila/f/florian.carichon/.conda/envs/work0/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/mila/f/florian.carichon/.conda/envs/work0/lib/python3.10/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Layer :  layer_original
Newness Human Mean :  0.0028484571437995872
Newness LLM Mean :  0.0079112853936784
Newness ttest:  0.007620622987283007
Newness utest : 0.2222783859267937
----
Uniq Human Mean :  0.41785444088924406
Uniq LLM Mean :  0.5387996758920184
Uniq ttest:  0.02603535129901908
Uniq utest : 0.03681350864696227
----
Diff Human Mean :  0.7547152980650726
Diff LLM Mean :  0.9944571175345944
Diff ttest:  nan
Diff utest : 0.11637222321064714
----
New Surp Human Mean :  0.7419202824306748
New Surp LLM Mean :  0.9725264050768997
New Surp ttest:  0.023363996186189024
New Surp  utest : 0.009055456664712636
----
Dist Surp Human Mean :  0.45504828824402965
Dist Surp LLM Mean :  0.5538503349305814
Dist Surp ttest:  0.04689223096382202
Dist Surp  utest : 0.031492367151090495




100%|██████████| 495/495 [19:04<00:00,  2.31s/it]
/home/mila/f/florian.carichon/.conda/envs/work0/lib/python3.10/site-packages/scipy/_lib/deprecation.py:234: SmallSampleWarning: After omitting NaNs, one or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  return f(*args, **kwargs)


Layer :  layer_1
Newness Human Mean :  0.0009419510265975663
Newness LLM Mean :  0.003409458833577338
Newness ttest:  0.004242358685776614
Newness utest : 0.013457526330308792
----
Uniq Human Mean :  0.5529131817855268
Uniq LLM Mean :  0.6516804164622823
Uniq ttest:  0.023685499535076135
Uniq utest : nan
----
Diff Human Mean :  0.9334716308238363
Diff LLM Mean :  0.9735222177944636
Diff ttest:  nan
Diff utest : 0.5190570648300906
----
New Surp Human Mean :  0.8181512920839206
New Surp LLM Mean :  0.9870442337451819
New Surp ttest:  0.026969456282643543
New Surp  utest : 0.012706562418355523
----
Dist Surp Human Mean :  0.28304356485809407
Dist Surp LLM Mean :  0.1953500109538496
Dist Surp ttest:  0.13361688863817162
Dist Surp  utest : 0.17255444477977644




...

Layer :  layer_-2
Newness Human Mean :  0.0008531328599506567
Newness LLM Mean :  0.0034962887824550853
Newness ttest:  0.0031419984852535596
Newness utest : 0.014831691197950773
----
Uniq Human Mean :  0.6679513104036536
Uniq LLM Mean :  0.6597347410385719
Uniq ttest:  0.29578763764284044
Uniq utest : nan
----
Diff Human Mean :  0.9408154211782239
Diff LLM Mean :  0.9683234249202478
Diff ttest:  nan
Diff utest : 0.6341576790455671
----
New Surp Human Mean :  0.8594606742419617
New Surp LLM Mean :  0.9833404893764212
New Surp ttest:  0.052407494327573925
New Surp  utest : 0.02398747642585134
----
Dist Surp Human Mean :  0.05399981771686312
Dist Surp LLM Mean :  0.1837505278748264
Dist Surp ttest:  0.056718440450921136
Dist Surp  utest : 0.09714053524378315




100%|██████████| 495/495 [18:50<00:00,  2.28s/it]


Layer :  layer_-1
Newness Human Mean :  0.0012704002154763233
Newness LLM Mean :  0.0041115722203914525
Newness ttest:  0.0032452282252366516
Newness utest : 0.0235358032602019
----
Uniq Human Mean :  0.513799606057617
Uniq LLM Mean :  0.5867567593704568
Uniq ttest:  0.03526542755222317
Uniq utest : nan
----
Diff Human Mean :  0.9257770067711099
Diff LLM Mean :  0.9715820028953117
Diff ttest:  nan
Diff utest : 0.44210794583542706
----
New Surp Human Mean :  0.8146929084744072
New Surp LLM Mean :  0.9723437140327339
New Surp ttest:  0.03263823753867098
New Surp  utest : 0.01588391947695492
----
Dist Surp Human Mean :  0.4170380136260847
Dist Surp LLM Mean :  0.4770221292107046
Dist Surp ttest:  0.25713332539093997
Dist Surp  utest : 0.2098873418001642




100%|██████████| 495/495 [19:05<00:00,  2.31s/it]


Layer :  layer_last
Newness Human Mean :  0.0019601264446765083
Newness LLM Mean :  0.005390251806595192
Newness ttest:  0.002691304093345355
Newness utest : 0.02429453926420589
----
Uniq Human Mean :  0.4121506927475228
Uniq LLM Mean :  0.5163350151037354
Uniq ttest:  0.03258382313621395
Uniq utest : nan
----
Diff Human Mean :  0.5707101333641506
Diff LLM Mean :  0.9341771612306022
Diff ttest:  nan
Diff utest : 0.10834389998628764
----
New Surp Human Mean :  0.6614846274550054
New Surp LLM Mean :  0.9007158627242343
New Surp ttest:  0.03063087751062413
New Surp  utest : 0.03334963116693107
----
Dist Surp Human Mean :  0.4328478801751277
Dist Surp LLM Mean :  0.5084743782123194
Dist Surp ttest:  0.13126613354487277
Dist Surp  utest : 0.08700148319302618




In [ ]:
## AGGREGATED VALUES AND T-TEST
countries = list(dict_values.keys())
gen_mean_x, gen_mean_y = [], []
gen_ttest_pvalue = []
gen_utest_pval = []
for country in countries:
    x = np.array(dict_values[country]['Human_new'])
    y = np.array(dict_values[country]['LLM_new'])

    # Step 1: Mean values
    gen_mean_x.append(np.mean(x))
    gen_mean_y.append(np.mean(y))

    # Step 2a: Independent t-test (Welch’s by default with equal_var=False)
    t_stat, p_val_t = stats.ttest_ind(x, y, equal_var=False, nan_policy="omit")
    if not np.isnan(p_val_t):
        gen_ttest_pvalue.append(p_val_t)

    # Step 2b: Mann–Whitney U test
    u_stat, p_val_u = stats.mannwhitneyu(x, y, alternative='two-sided')
    gen_utest_pval.append(p_val_u)

In [123]:
print(np.mean(gen_mean_x))
print(np.mean(gen_mean_y))
print(np.mean(gen_ttest_pvalue))
print(np.mean(gen_utest_pval))

0.002805420252838654
0.0046722702810057795
0.022089559340476506
0.20241767304864186
